# Extract the Codebook HTML into a dataframe

This notebook is capable of converting a local downloaded version of the ISARIC Codebook (save as single web page) into a dataframe containg the:
ID, Variable / Field Name, Field Label, Field Attributes, Dictionary (option value and label)

Usage:
- Save the ISARIC Codebook as Single web page locally
- Put in same folder as this Notebook
- Run

In [1]:
import re
import pandas as pd
from tabulate import tabulate
from bs4 import BeautifulSoup                       # Beautiful Soup Version 4
RA_Works = 0                                        # Initiate the Count

# location of the download webpage containing the ISARIC Codebook
url = './ISARIC COVID-19 CORE & Follow-up _ REDCap.html'

# Read and create a soup from the url
f = open(url, 'r')
s = f.read()
soup = BeautifulSoup(s,"html")


# define the column names of the codebook
columns = ['id', 'variable', 'label', 'type', 'dict']

# create the dataframe for the codebook
df_codebook = pd.DataFrame(index=[], columns=columns)

# soupify the lot
tables = soup.find_all('table')

# it appears the Codebook is the second table
table_body = tables[1].find('tbody')

# find all the rows in the table of the Codebook
rows = table_body.find_all('tr')

# counter to make sure the correct rows are identified based on the '#' in the Codebook
x=1
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    try:
        if int(cols[0]) == x:
            new_id = cols[0]
            new_variable = cols[1].split('\n',1)[0]
            new_label = cols[2]
            new_type = re.split('\d+', cols[3])[0]
            new_dict = {}
            try:
                lst = cols[4:]
                new_dict = {lst[i]: lst[i+1] for i in range(0, len(lst),2)}
            except:
                pass
            new_item = pd.Series([new_id, new_variable, new_label, new_type, new_dict], index=columns)
            df_codebook = df_codebook.append(new_item, ignore_index=True)
            x=x+1
    except:
        pass

# make a copy of df_codebook to display the codebook in readable format
# for this 'label', 'type', 'dict' must be maximized to contain n characters
n = 25
df_codebook_temp = df_codebook.copy()
df_codebook_temp['label'] = df_codebook_temp['label'].str[:n]
df_codebook_temp['type'] = df_codebook_temp['type'].str[:n]
df_codebook_temp['dict'] = df_codebook_temp['dict'].astype(str)
df_codebook_temp['dict'] = df_codebook_temp['dict'].str[:n]
    
print(tabulate(df_codebook_temp, columns, tablefmt='rst'))#, tablefmt="github"))

====  ====  ========================================================  ==========================  =========================  =========================
  ..    id  variable                                                  label                       type                       dict
====  ====  ========================================================  ==========================  =========================  =========================
   0     1  subjid                                                    Participant Identificatio  text                       {}
   1     2  participant_identification_number_pin_complete            Section Header: Form Stat  dropdown                   {'0': 'Incomplete', '1':
   2     3  tos_v2                                                    Please click on the PDF d  descriptive                {}
   3     4  tos_facat_v2                                              There are two mechanisms   radio                      {'1': 'Via electronic sig
   4     5  nov